In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, KFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model, clone_model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, MaxPooling1D,
    Dropout, Flatten, Dense, LSTM, GRU
)
from tensorflow.keras.callbacks import EarslyStopping, ReduceLROnPlateau
from sklearn.ensemble import RandomForestRegressor
import joblib
import xgboost as xgb
from tensorflow.keras import regularizers
from scipy import stats
from scipy.stats import ks_2samp, anderson_ksamp

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

2025-12-01 01:38:45.453510: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 01:38:47.009149: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 01:38:53.235193: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import json
with open('preprocessing_metadata.json', 'r') as f:
    metadata = json.load(f)
features = metadata['predictor_features']

target_col = 'ppp'
depth_col = 'tvd'

features

['tvd',
 'dt',
 'dt_nct',
 'gr',
 'sphi',
 'hp',
 'ob',
 'rhob_combined',
 'res_deep',
 'eaton_ratio',
 'hp_gradient',
 'ob_gradient',
 'tvd_normalized']

In [4]:
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
train_val_combined = pd.concat([train_df, val_df], ignore_index=True)

blind_df = pd.read_csv('test_data.csv')

X_train_val = train_val_combined[features].values
y_train_val = train_val_combined[target_col].values

X_test = blind_df[features].values
y_test = blind_df[target_col].values

print(f"\nData sizes:")
print(f"  Train+Val combined: {len(X_train_val)} (for CV stacking)")
print(f"  Test: {len(X_test)} (for final evaluation)")


Data sizes:
  Train+Val combined: 188631 (for CV stacking)
  Test: 84973 (for final evaluation)


In [5]:
# Callbacks
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)

In [6]:
print("\n" + "="*60)
print("STEP 1: GENERATING OUT-OF-FOLD PREDICTIONS")
print("="*60)

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Storage for out-of-fold predictions
oof_preds = {
    'CNN': np.zeros(len(X_train_val)),
    'DFNN': np.zeros(len(X_train_val)),
    'RNN': np.zeros(len(X_train_val)),
    'RF': np.zeros(len(X_train_val)),
    'XGBoost': np.zeros(len(X_train_val))
}

# Storage for final models (trained on all data)
final_models = {}

print(f"\nPerforming {n_folds}-fold cross-validation...")
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
    print(f"\nFold {fold + 1}/{n_folds}...")
    
    # Split data
    X_fold_train = X_train_val[train_idx]
    y_fold_train = y_train_val[train_idx]
    X_fold_val = X_train_val[val_idx]
    y_fold_val = y_train_val[val_idx]
    
    # Scale
    fold_scaler = StandardScaler()
    X_fold_train_s = fold_scaler.fit_transform(X_fold_train)
    X_fold_val_s = fold_scaler.transform(X_fold_val)
    
    # Reshape for CNN/RNN
    X_fold_train_cnn = X_fold_train_s.reshape(-1, len(features), 1)
    X_fold_val_cnn = X_fold_val_s.reshape(-1, len(features), 1)
    X_fold_train_rnn = X_fold_train_s.reshape(-1, len(features), 1)
    X_fold_val_rnn = X_fold_val_s.reshape(-1, len(features), 1)
    
    # Train CNN
    cnn_fold = Sequential([
        Input((len(features),1)),
        Conv1D(64, kernel_size=2, padding='same', activation='relu',
               kernel_regularizer=regularizers.L2(0.001)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Conv1D(128, kernel_size=2, padding='same', activation='relu',
               kernel_regularizer=regularizers.L2(0.001)),
        BatchNormalization(),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.002)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.002)),
        BatchNormalization(), 
        Dropout(0.3),
        Dense(1)
    ])
    cnn_fold.compile(optimizer='adam', loss='mse')
    cnn_fold.fit(X_fold_train_cnn, y_fold_train,
                 validation_split=0.1, epochs=100, batch_size=32,
                 callbacks=[es, rlr], verbose=0)
    oof_preds['CNN'][val_idx] = cnn_fold.predict(X_fold_val_cnn, verbose=0).flatten()
    
    # Train DFNN
    dfnn_fold = Sequential([
        Dense(128, activation='relu', input_shape=(len(features),)),
        BatchNormalization(),
        Dropout(0.15),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.15),
        Dense(32, activation='relu'),
        Dropout(0.1),
        Dense(1)
    ])
    dfnn_fold.compile(optimizer='adam', loss='mse')
    dfnn_fold.fit(X_fold_train_s, y_fold_train,
                  validation_split=0.1, epochs=100, batch_size=64,
                  callbacks=[es, rlr], verbose=0)
    oof_preds['DFNN'][val_idx] = dfnn_fold.predict(X_fold_val_s, verbose=0).flatten()
    
    # Train RNN
    rnn_fold = Sequential([
        Input((len(features), 1)),
        LSTM(64, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1)
    ])
    rnn_fold.compile(optimizer='adam', loss='mse')
    rnn_fold.fit(X_fold_train_rnn, y_fold_train,
                 validation_split=0.1, epochs=130, batch_size=16,
                 callbacks=[es, rlr], verbose=0)
    oof_preds['RNN'][val_idx] = rnn_fold.predict(X_fold_val_rnn, verbose=0).flatten()
    
    # Train RF
    rf_fold = RandomForestRegressor(
        n_estimators=500, max_depth=15,
        min_samples_split=5, min_samples_leaf=2,
        random_state=42, n_jobs=-1
    )
    rf_fold.fit(X_fold_train_s, y_fold_train)
    oof_preds['RF'][val_idx] = rf_fold.predict(X_fold_val_s)
    
    # Train XGBoost
    xgb_fold = xgb.XGBRegressor(
        n_estimators=1000, max_depth=6,
        learning_rate=0.1, subsample=0.8,
        colsample_bytree=0.8, random_state=42,
        early_stopping_rounds=10
    )
    xgb_fold.fit(X_fold_train_s, y_fold_train,
                 eval_set=[(X_fold_val_s, y_fold_val)],
                 verbose=False)
    oof_preds['XGBoost'][val_idx] = xgb_fold.predict(X_fold_val_s)

print("\n✓ Out-of-fold predictions generated!")


STEP 1: GENERATING OUT-OF-FOLD PREDICTIONS

Performing 5-fold cross-validation...

Fold 1/5...


I0000 00:00:1764571138.283071  676035 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 140842 MB memory:  -> device: 0, name: NVIDIA H200, pci bus id: 0000:4c:00.0, compute capability: 9.0
2025-12-01 01:39:02.134117: I external/local_xla/xla/service/service.cc:163] XLA service 0x146f80014190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-01 01:39:02.134134: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA H200, Compute Capability 9.0
2025-12-01 01:39:02.220231: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-01 01:39:02.554640: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400
2025-12-01 01:39:02.782843: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because


Fold 2/5...


2025-12-01 02:06:32.706693: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-01 02:06:32.706719: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-01 02:06:32.910975: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_100', 4 bytes spill stores, 4 bytes spill loads

2025-12-01 02:06:33.473135: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Re


Fold 3/5...


...


Fold 4/5...


2025-12-01 02:32:44.668939: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 8 bytes spill stores, 8 bytes spill loads




Fold 5/5...


2025-12-01 02:49:24.668525: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 8 bytes spill stores, 8 bytes spill loads




✓ Out-of-fold predictions generated!


In [7]:
print("\n" + "="*60)
print("STEP 2: TRAINING META-MODEL ON OUT-OF-FOLD PREDICTIONS")
print("="*60)

# Create feature matrix for meta-model
X_meta = np.column_stack([
    oof_preds['CNN'],
    oof_preds['DFNN'],
    oof_preds['RNN'],
    oof_preds['RF'],
    oof_preds['XGBoost']
])

models = {
    'CNN': cnn_fold,
    'DFNN': dfnn_fold,
    'RNN': rnn_fold,
    'Random Forest': rf_fold,
    'XGBoost': xgb_fold
}

# Train stacking model
stacking_model = Ridge(alpha=1.0)
stacking_model.fit(X_meta, y_train_val)

print("\nMeta-model trained!")
print("Ridge coefficients:")
for name, coef in zip(models, stacking_model.coef_):
    print(f"  {name:15}: {coef:.4f}")


STEP 2: TRAINING META-MODEL ON OUT-OF-FOLD PREDICTIONS

Meta-model trained!
Ridge coefficients:
  CNN            : -0.0480
  DFNN           : 0.0009
  RNN            : -0.0667
  Random Forest  : 0.5436
  XGBoost        : 0.5664


In [8]:
print("\n" + "="*60)
print("STEP 3: RETRAINING BASE MODELS ON ALL TRAIN+VAL DATA")
print("="*60)

# Scale all data
scaler_final = StandardScaler()
X_train_val_s = scaler_final.fit_transform(X_train_val)
X_test_s = scaler_final.transform(X_test)

# Reshape
X_train_val_cnn = X_train_val_s.reshape(-1, len(features), 1)
X_test_cnn = X_test_s.reshape(-1, len(features), 1)
X_train_val_rnn = X_train_val_s.reshape(-1, len(features), 1)
X_test_rnn = X_test_s.reshape(-1, len(features), 1)

print("Retraining all models on full dataset...")

# CNN
print("  CNN...")
cnn_final = Sequential([
    Input((len(features),1)),
    Conv1D(64, kernel_size=2, padding='same', activation='relu',
           kernel_regularizer=regularizers.L2(0.001)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    Conv1D(128, kernel_size=2, padding='same', activation='relu',
           kernel_regularizer=regularizers.L2(0.001)),
    BatchNormalization(),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.002)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0.002)),
    BatchNormalization(), 
    Dropout(0.3),
    Dense(1)
])
cnn_final.compile(optimizer='adam', loss='mse')
cnn_final.fit(X_train_val_cnn, y_train_val,
              validation_split=0.1, epochs=100, batch_size=32,
              callbacks=[es, rlr], verbose=0)

# DFNN
print("  DFNN...")
dfnn_final = Sequential([
    Dense(128, activation='relu', input_shape=(len(features),)),
    BatchNormalization(),
    Dropout(0.15),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.15),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(1)
])
dfnn_final.compile(optimizer='adam', loss='mse')
dfnn_final.fit(X_train_val_s, y_train_val,
               validation_split=0.1, epochs=100, batch_size=64,
               callbacks=[es, rlr], verbose=0)

# RNN
print("  RNN...")
rnn_final = Sequential([
    Input((len(features), 1)),
    LSTM(64, return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(32, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1)
])
rnn_final.compile(optimizer='adam', loss='mse')
rnn_final.fit(X_train_val_rnn, y_train_val,
              validation_split=0.1, epochs=130, batch_size=16,
              callbacks=[es, rlr], verbose=0)

# RF
print("  Random Forest...")
rf_final = RandomForestRegressor(
    n_estimators=500, max_depth=15,
    min_samples_split=5, min_samples_leaf=2,
    random_state=42, n_jobs=-1
)
rf_final.fit(X_train_val_s, y_train_val)

# XGBoost
print("  XGBoost...")
xgb_final = xgb.XGBRegressor(
    n_estimators=1000, max_depth=6,
    learning_rate=0.1, subsample=0.8,
    colsample_bytree=0.8, random_state=42
)
xgb_final.fit(X_train_val_s, y_train_val, verbose=False)

print("\n✓ All models retrained on full dataset!")


STEP 3: RETRAINING BASE MODELS ON ALL TRAIN+VAL DATA
Retraining all models on full dataset...
  CNN...
  DFNN...


2025-12-01 03:01:10.832444: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 8 bytes spill stores, 8 bytes spill loads

2025-12-01 03:01:14.043000: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-01 03:01:14.043051: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-01 03:01:14.043083: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All co

  RNN...
  Random Forest...
  XGBoost...

✓ All models retrained on full dataset!


In [9]:
print("\n" + "="*60)
print("STEP 4: FINAL EVALUATION ON TEST SET")
print("="*60)

def evaluate(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    rel = rmse/(y_true.max()-y_true.min())*100
    print(f"{name}: R²={r2:.4f}, RMSE={rmse:.2f}, MAE={mae:.2f}, RelRMSE={rel:.2f}%")

y_test_pred_cnn = cnn_final.predict(X_test_cnn, verbose=0).flatten()
y_test_pred_dfnn = dfnn_final.predict(X_test_s, verbose=0).flatten()
y_test_pred_rnn = rnn_final.predict(X_test_rnn, verbose=0).flatten()
y_test_pred_rf = rf_final.predict(X_test_s)
y_test_pred_xgb = xgb_final.predict(X_test_s)

print("\nIndividual model results:")
evaluate("CNN", y_test, y_test_pred_cnn)
evaluate("DFNN", y_test, y_test_pred_dfnn)
evaluate("RNN", y_test, y_test_pred_rnn)
evaluate("Random Forest", y_test, y_test_pred_rf)
evaluate("XGBoost", y_test, y_test_pred_xgb)

# Stacking prediction
X_test_meta = np.column_stack([
    y_test_pred_cnn,
    y_test_pred_dfnn,
    y_test_pred_rnn,
    y_test_pred_rf,
    y_test_pred_xgb
])
y_test_pred_stacking = stacking_model.predict(X_test_meta)

print("\nEnsemble results:")
y_test_pred_simple = np.mean(X_test_meta, axis=1)
evaluate("Simple Average", y_test, y_test_pred_simple)
evaluate("Stacking (CV-based, PROPER)", y_test, y_test_pred_stacking)


STEP 4: FINAL EVALUATION ON TEST SET

Individual model results:
CNN: R²=0.8938, RMSE=610.65, MAE=448.71, RelRMSE=6.61%
DFNN: R²=0.7852, RMSE=868.28, MAE=629.59, RelRMSE=9.40%
RNN: R²=0.8835, RMSE=639.61, MAE=459.56, RelRMSE=6.92%
Random Forest: R²=0.8578, RMSE=706.44, MAE=544.34, RelRMSE=7.64%
XGBoost: R²=0.8368, RMSE=756.94, MAE=563.73, RelRMSE=8.19%

Ensemble results:
Simple Average: R²=0.8894, RMSE=623.04, MAE=463.02, RelRMSE=6.74%
Stacking (CV-based, PROPER): R²=0.8490, RMSE=728.05, MAE=557.82, RelRMSE=7.88%
